In [409]:
# -*- coding: utf-8 -*-
# from db_users import *
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

sns.set_style('white')

import scipy
import sys
from math import sqrt

from scipy.spatial.distance import cdist,pdist, squareform
from sklearn.metrics.pairwise import euclidean_distances

In [55]:
column_name = ['user_id', 'item_id', 'rating']
data_frame = pd.read_csv("u.data", sep='\t', names=column_name, usecols=column_name)

In [56]:
column_name = ['item_id', 'title']
movie_title = pd.read_csv("u.item", sep='|', names=column_name, usecols=column_name)

In [57]:
data_frame = pd.merge(data_frame, movie_title, on='item_id')
data_frame.head()

,user_id,item_id,rating,title
0,196,242,3,Kolya (1996)
1,63,242,3,Kolya (1996)
2,226,242,5,Kolya (1996)
3,154,242,3,Kolya (1996)
4,306,242,5,Kolya (1996)


In [60]:
data_frame.groupby('title')['rating'].mean().sort_values(ascending=False).head()
#data_frame.groupby('title')['rating'].count().sort_values(ascending=False).head(10)

title
Third Man, The (1949)                                                                5.0
Old Lady Who Walked in the Sea, The (Vieille qui marchait dans la mer, La) (1991)    5.0
Firestorm (1998)                                                                     5.0
Night on Earth (1991)                                                                5.0
Gay Divorcee, The (1934)                                                             5.0
Name: rating, dtype: float64

In [61]:
ratings = pd.DataFrame( data_frame.groupby('title')['rating'].mean())

ratings['count'] = data_frame.groupby('title')['rating'].count()

In [62]:
ratings = ratings.sort_values(by=['count'], ascending=False)

ratings.head()

,rating,count
title,,
Contact (1997),3.971429,35
Star Wars (1977),4.379310,29
Raiders of the Lost Ark (1981),4.148148,27
"Silence of the Lambs, The (1991)",4.230769,26
Fargo (1996),4.080000,25


In [568]:
movie_matrix = data_frame.pivot_table(index='user_id',columns='title',values='rating')
movie_matrix.head()

title,101 Dalmatians (1996),12 Angry Men (1957),187 (1997),2 Days in the Valley (1996),"20,000 Leagues Under the Sea (1954)",2001: A Space Odyssey (1968),3 Ninjas: High Noon At Mega Mountain (1998),"39 Steps, The (1935)",8 1/2 (1963),Absolute Power (1997),...,"Women, The (1939)",Wonderland (1997),"Wrong Trousers, The (1993)",Wyatt Earp (1994),Year of the Horse (1997),Young Frankenstein (1974),Young Guns (1988),"Young Poisoner's Handbook, The (1995)",Zeus and Roxanne (1997),unknown
user_id,,,,,,,,,,,,,,,,,,,,,
1,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0


In [554]:
def euclidiana(base,user1,user2): #faz o calculo de aproximidade dos usuários
    dados = {}

    for item in base[user1]:
        if item in base[user2]:
            dados[item] = 1

    if len(dados) == 0:
        return 0

    soma = sum ([pow(base[user1][item] - base[user2][item],2)  for item in base[user1] if item in base[user2]])
    return 1/(1+sqrt(soma))

In [569]:
f_u_rating = movie_matrix['unknown']
f_u_rating.dropna(inplace=True)
f_u_rating

user_id
5      4.0
130    5.0
Name: unknown, dtype: float64

In [538]:
dist = pdist(movie_matrix[movie_matrix.columns], 'euclidean')
df_dist = pd.DataFrame(squareform(dist))# matriz de similaridade

In [539]:
base = movie_matrix.fillna(0)
result = { }

for idx in f_u_rating.index: # busca user que deu rating no filme
    print(idx)
    filmes = movie_matrix.iloc[idx].dropna() #seleciona todos os filmes desse user
#     print(filmes)
    for name,rating in filmes.iteritems():#pega todos os nomes dos filmes desse user novo
        
        user = name
        
        similaridade = [(euclidiana(base,user, outro), outro) for outro in base if outro != user]
#         print( len(similaridade) )
        similaridade.sort()
        similaridade.reverse()
        
        result[idx] = similaridade[0:45]
        
        
#         print( similaridade )
#         print(name,movie_matrix[name].dropna())
        #sys.exit()


In [342]:
result

{37: [(1.0, 'Zeus and Roxanne (1997)'),
  (1.0, "Young Poisoner's Handbook, The (1995)"),
  (1.0, 'Young Guns (1988)'),
  (1.0, 'Young Frankenstein (1974)'),
  (1.0, 'Year of the Horse (1997)'),
  (1.0, 'Wyatt Earp (1994)'),
  (1.0, 'Wrong Trousers, The (1993)'),
  (1.0, 'Wonderland (1997)'),
  (1.0, 'Women, The (1939)'),
  (1.0, 'Wolf (1994)'),
  (1.0, 'Wizard of Oz, The (1939)'),
  (1.0, 'With Honors (1994)'),
  (1.0, 'Winnie the Pooh and the Blustery Day (1968)'),
  (1.0, 'Wings of the Dove, The (1997)'),
  (1.0, 'Wings of Desire (1987)'),
  (1.0, 'Window to Paris (1994)'),
  (1.0, 'Willy Wonka and the Chocolate Factory (1971)'),
  (1.0, "William Shakespeare's Romeo and Juliet (1996)"),
  (1.0, 'Wild Bunch, The (1969)'),
  (1.0, 'Wild Bill (1995)'),
  (1.0, "Widows' Peak (1994)"),
  (1.0, 'Whole Wide World, The (1996)'),
  (1.0, 'White Squall (1996)'),
  (1.0, 'While You Were Sleeping (1995)'),
  (1.0, 'When the Cats Away (Chacun cherche son chat) (1996)'),
  (1.0, 'When We Were Kin

In [454]:
teste_matrix = data_frame.pivot_table(index='title',columns='user_id',values='rating')
X_testing = np.asarray(teste_matrix.fillna(0))
X_testing = pd.DataFrame(X_testing)

data = pd.DataFrame( euclidean_distances(X_testing,X_testing) ) 
data

,0,1,2,3,4,5,6,7,8,9,...,1044,1045,1046,1047,1048,1049,1050,1051,1052,1053
0,0.000000,12.845233,7.615773,9.110434,9.110434,18.193405,7.141428,11.135529,9.949874,10.535654,...,7.615773,8.062258,12.000000,9.486833,8.602325,13.038405,8.660254,8.602325,7.615773,8.602325
1,12.845233,0.000000,11.180340,11.489125,12.247449,19.949937,10.488088,13.820275,10.770330,14.212670,...,11.180340,10.000000,14.525839,10.816654,11.874342,14.525839,11.916375,11.874342,11.180340,12.529964
2,7.615773,11.180340,0.000000,6.557439,6.557439,17.058722,3.316625,9.165151,7.681146,8.774964,...,4.242641,5.000000,10.198039,7.071068,5.830952,12.409674,5.916080,5.830952,4.242641,7.071068
3,9.110434,11.489125,6.557439,0.000000,8.246211,17.776389,6.000000,10.440307,9.165151,10.954451,...,6.557439,7.071068,11.357817,8.660254,7.681146,13.379088,7.745967,7.681146,6.557439,8.660254
4,9.110434,12.247449,6.557439,8.246211,0.000000,17.776389,6.000000,10.440307,9.165151,10.954451,...,5.000000,7.071068,11.357817,8.660254,7.681146,13.379088,7.745967,7.681146,6.557439,8.660254
5,18.193405,19.949937,17.058722,17.776389,17.776389,0.000000,16.852300,18.894444,18.220867,19.183326,...,17.058722,17.262677,19.416488,17.972201,17.521415,20.223748,17.549929,17.521415,17.058722,17.972201
6,7.141428,10.488088,3.316625,6.000000,6.000000,16.852300,0.000000,8.774964,7.211103,9.380832,...,3.316625,3.162278,9.848858,5.744563,5.196152,12.124356,5.291503,5.196152,3.316625,6.557439
7,11.135529,13.820275,9.165151,10.440307,10.440307,18.894444,8.774964,0.000000,11.180340,12.688578,...,9.165151,9.539392,13.038405,10.770330,10.000000,13.416408,10.049876,10.000000,9.165151,10.770330
8,9.949874,10.770330,7.681146,9.165151,9.165151,18.220867,7.211103,11.180340,0.000000,11.661904,...,7.681146,8.124038,12.041595,9.539392,8.660254,12.041595,8.717798,8.660254,7.681146,9.539392
9,10.535654,14.212670,8.774964,10.954451,10.954451,19.183326,9.380832,12.688578,11.661904,0.000000,...,9.746794,10.099505,13.453624,11.269428,10.535654,15.198684,10.583005,10.535654,9.746794,11.269428


In [463]:
for idx in f_u_rating.index: # busca user que deu rating no filme
    print(idx)
    userRated = movie_matrix.iloc[idx].dropna() #seleciona todos os filmes desse user
    
    print( type(userRated) )
    

37
<class 'pandas.core.series.Series'>
92
<class 'pandas.core.series.Series'>
110
<class 'pandas.core.series.Series'>
217
<class 'pandas.core.series.Series'>
268
<class 'pandas.core.series.Series'>
280
<class 'pandas.core.series.Series'>


In [583]:
data[[0]].sort_values( by=0,ascending=True)[:5] # pega a menor distancia entre os outros filmes

,0
0,0.000000
547,5.656854
741,6.164414
459,6.164414
35,6.164414
